In [5]:
from sklearn.datasets import load_boston

data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

variable = "LSTAT"
x = df[variable].values
y = data.target


# 데이터 불러오기

In [1]:
import pandas as pd

columns_list = []

for i in range(0, 21):
    columns_list.append(i)

print(columns_list)


data_test = pd.read_csv('../data/BankChurners.csv', usecols=columns_list)
data_test.head()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


###  Attrition_Flag 값 변환

In [2]:
data_test['Attrition_Flag'].replace("Existing Customer", "0", inplace=True)
data_test['Attrition_Flag'].replace("Attrited Customer", "1", inplace=True)

data_test['Attrition_Flag'] = pd.to_numeric(data_test['Attrition_Flag'])

In [74]:
variable = 'Credit_Limit'

In [75]:
test_col = data_test[variable].values
target_col = data_test['Attrition_Flag'].values

# optbinning

In [76]:
from optbinning import ContinuousOptimalBinning

optb = ContinuousOptimalBinning(name=variable, dtype="numerical")

In [77]:
optb.fit(test_col, target_col)

ContinuousOptimalBinning(name='Credit_Limit')

### binning table

In [78]:
binning_table = optb.binning_table

In [79]:
binning_table.build()

,Bin,Count,Count (%),Sum,Std,Mean,Min,Max,Zeros count,WoE,IV
0,"(-inf, 1438.65)",507,0.050064,124.0,0.429835,0.244576,0.0,1.0,383,0.083916,0.004201
1,"[1438.65, 2801.50)",2576,0.254370,432.0,0.373601,0.167702,0.0,1.0,2144,0.007042,0.001791
2,"[2801.50, 5061.50)",2310,0.228103,378.0,0.369945,0.163636,0.0,1.0,1932,0.002977,0.000679
3,"[5061.50, 9130.00)",1718,0.169646,257.0,0.356672,0.149593,0.0,1.0,1461,-0.011067,0.001877
4,"[9130.00, 12047.50)",700,0.069122,93.0,0.33942,0.132857,0.0,1.0,607,-0.027802,0.001922
5,"[12047.50, 22381.00)",1244,0.122840,171.0,0.344332,0.137460,0.0,1.0,1073,-0.0232,0.002850
6,"[22381.00, 32814.50)",515,0.050854,74.0,0.350774,0.143689,0.0,1.0,441,-0.01697,0.000863
7,"[32814.50, inf)",557,0.055001,98.0,0.380771,0.175943,0.0,1.0,459,0.015283,0.000841
8,Special,0,0.000000,0.0,NaN,0.000000,NaN,NaN,0,-0.16066,0.000000
9,Missing,0,0.000000,0.0,NaN,0.000000,NaN,NaN,0,-0.16066,0.000000


In [80]:
test_bin = optb.transform(data_test['Credit_Limit'], metric="bins")
opt_col= pd.Series(test_bin)

In [81]:
opt_col

0        [12047.50, 22381.00)
1          [5061.50, 9130.00)
2          [2801.50, 5061.50)
3          [2801.50, 5061.50)
4          [2801.50, 5061.50)
                 ...         
10122      [2801.50, 5061.50)
10123      [2801.50, 5061.50)
10124      [5061.50, 9130.00)
10125      [5061.50, 9130.00)
10126     [9130.00, 12047.50)
Length: 10127, dtype: object

# 함수로 변환 

In [59]:
from optbinning import ContinuousOptimalBinning

def opt_col(test_col, target_col):
    
    optb = ContinuousOptimalBinning(name=variable, dtype="numerical")
    
    optb.fit(test_col, target_col)
    
    binning_table = optb.binning_table
    
    binning_table.build()
    
    test_bin = optb.transform(test_col, metric="bins")
    opt_complete = pd.Series(test_bin)
    
    return opt_complete

In [62]:
test_avg = opt_col(test_col, target_col)